In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import csv

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [3]:
lenc = LabelEncoder()

In [4]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem.wordnet import WordNetLemmatizer

In [5]:
snow = SnowballStemmer('english')
wnlem = WordNetLemmatizer()

In [6]:
class PreProcessor:
    def __init__(self, raw_list):
        self.raw_list = raw_list
        self.lowercased = None
        self.sent_tokenized = None
        self.word_tokenized = None
        self.cleaned_special_chars = None
        self.without_stopwords = None
        self.stemmed = None
        self.lemmed = None

    def lowercase(self):
        lowercased = []
        for text in self.raw_list:
            lowercased.append(text.lower())
        self.lowercased = lowercased
        return self.lowercased

    def tokenize_sentence(self):
        self.sent_tokenized = [sent_tokenize(sent) for sent in self.lowercased]
        return self.sent_tokenized
    
    def tokenize_word(self):
        self.word_tokenized = [word_tokenize(word) for word in self.lowercased]
        return self.word_tokenized

    def clean_special_chars(self):
        cleaned_text = []
        for sentence in self.sent_tokenized:
          cleaned_sent = []
          for word in sentence:
            clean = re.sub(r'[^\w\s]', "", word)
            if clean != "":
              cleaned_sent.append(clean)
          cleaned_text.append(cleaned_sent)
        self.cleaned_special_chars = cleaned_text
        return self.cleaned_special_chars


    def remove_stopwords(self):
        without_stopwords = []
        for sentences in self.cleaned_special_chars:
            no_stop = []
            for words in sentences:
              for word in words.split(" "):
                if word not in stopwords.words('english'):
                  no_stop.append(word)
            without_stopwords.append(" ".join(no_stop))
        self.without_stopwords = without_stopwords
        return self.without_stopwords
    
    def stem(self):
        stemmed_list = []
        for elem in [i for i in self.without_stopwords if type(i)!=list]:
          stemmed_list.append([snow.stem(word) for word in elem.split(" ")])
        self.stemmed = stemmed_list
        return self.stemmed
    
    def lemmatize(self):
        lemmed = []
        for item in [i for i in self.without_stopwords if type(i)!=list]:
          lemmed.append([wnlem.lemmatize(word) for word in item.split(" ")])
        self.lemmed = lemmed
        return self.lemmed

    def preprocess(self):
        self.lowercase()
        self.tokenize_sentence()
        self.tokenize_word()
        self.clean_special_chars()
        self.remove_stopwords()
        self.stem()
        self.lemmatize()


In [8]:
lyric_df = pd.read_csv('traincommasnewlinesremoved.csv',  usecols=range(0,5), header = None, delimiter=",", quoting=csv.QUOTE_NONE, 
                       encoding='utf-8')
lyric_df.columns = ['artist', 'song', 'genre', 'lang', 'lyrics']

In [ ]:
lyric_df.shape
lyric_df.head

In [10]:
# starting to standardise the column numbers here, 
#will get the longest lyrics (pink floyd - the wall, it seems) and pad the rest with zeros to be same length
lyric_df['lyrics']=lyric_df['lyrics'].fillna("")
lyrics = lyric_df['lyrics'].tolist()
max_lyric_len = max([i.split(" ") for i in lyrics], key=len)

In [11]:
word_col_num = len(max_lyric_len) #longest lyric 

In [ ]:
padded_split_lyrics = []
# uses too much ram going thru in one go
for index, row in lyric_df.iloc[:100000].iterrows():
  padded_list = [0] * word_col_num 
  list_of_lyrics = row['lyrics'].split(" ")
  padded_list[:len(list_of_lyrics)] = list_of_lyrics
  padded_split_lyrics.append(padded_list)
for index, row in lyric_df.iloc[100000:200000].iterrows():
  padded_list = [0] * word_col_num 
  list_of_lyrics = row['lyrics'].split(" ")
  padded_list[:len(list_of_lyrics)] = list_of_lyrics
  padded_split_lyrics.append(padded_list)
for index, row in lyric_df.iloc[200000:].iterrows():
  padded_list = [0] * word_col_num 
  list_of_lyrics = row['lyrics'].split(" ")
  padded_list[:len(list_of_lyrics)] = list_of_lyrics
  padded_split_lyrics.append(padded_list)
print(padded_split_lyrics[-10:]) 

In [ ]:
# adding the split words as columns to the dataframe
lyric_cols_array = np.array(padded_split_lyrics)
print(lyric_cols_array[:,:10])
lyric_cols_array = lyric_cols_array.transpose()
print(lyric_cols_array[0])
# lol i guess some of the lyrics are guitar tabs? can maybe filter those out

In [ ]:
for c,v in enumerate(lyric_cols_array):
  lyric_df['lyrics'+str(c)] = v.tolist()

lyric_df.lyrics0.head

In [ ]:
lyric_df = lyric_df.astype(str)
print(lyric_df['genre'])

In [ ]:
# not preprocessing atm, need to remove stopwords at least, or may end up using only key words

In [47]:

encoded_data = lyric_df.apply(lenc.fit_transform, axis=0)


In [50]:
encoded_df = encoded_data.values
# leaving out for now i think

In [52]:
# assigning lyrics and genre to info and class for later use, lyrics will become everything past the other columns rather than a single one
lyric_info = encoded_df[:, 5:]
lyric_classes = encoded_df[:, 2]

In [53]:
X_train,  X_test, y_train, y_test = train_test_split(
    lyric_info, lyric_classes, test_size=0.20, random_state=75
    )

In [ ]:
X_train
type(X_test)

In [36]:
def get_prior_prob(y, label):
  total = y.shape[0]
  actual = np.sum(y == label)

  return total / actual


In [37]:
def conditional_prob(X_train, y_train, feature_col, feature_val, label):
  X_filtered = X_train[y_train == label]
  num = np.sum(X_filtered[:, feature_col] == feature_val)
  denom = X_filtered.shape[0]

  return num/denom


In [38]:
def predict(X_train, y_train, X_test):
  classes = np.unique(y_train)
  features = X_train.shape[1]
  
  posterior_prob = []

  for label in classes:
    chance = 1.0
    for feature in range(features):
      cond = conditional_prob(X_train, y_train, feature, X_test[feature], label)
      chance = chance * cond 
    prior = get_prior_prob(y_train, label)
    posterior = chance * prior
    posterior_prob.append(posterior)

    most_likely = np.argmax(posterior_prob)

    return most_likely

In [43]:
def get_accuracy(X_train, y_train, X_test, y_test):
  preds = []
  for i in range (X_test.shape[0]):
    pred = predict(X_train, y_train, X_test[i])
    preds.append(pred)
  class_preds = np.array(preds)
  
  accuracy = np.sum(class_preds == y_test)/ class_preds.shape[0]

  return accuracy 


In [55]:
acc = get_accuracy(X_train, y_train, X_test, y_test)

In [ ]:
print(acc)